In [0]:
!wget -c https://github.com/roman-rybalko/telegram-data-clustering-contest/archive/master.zip -O master.zip
!unzip -u -q master.zip
!rm -Rf dataset1
!mv -v */dataset1 ./

--2019-11-17 17:05:59--  https://github.com/roman-rybalko/telegram-data-clustering-contest/archive/master.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/roman-rybalko/telegram-data-clustering-contest/zip/master [following]
--2019-11-17 17:05:59--  https://codeload.github.com/roman-rybalko/telegram-data-clustering-contest/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [               <=>  ] 337.40M  17.3MB/s    in 31s     

2019-11-17 17:06:30 (10.9 MB/s) - ‘master.zip’ saved [353785379]

renamed 'telegram-data-clustering-contest-master/dataset1' -> './dataset1'


In [7]:
import os
def get_files(d):
  return [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(d)) for f in fn]
files = [f for f in get_files('dataset1') if os.path.splitext(f)[1] == '.html']
len(files)

166551

In [11]:
!rm -vf gcp-auth*
!wget http://host.romanr.info/gcp-auth.json
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gcp-auth.json'

removed 'gcp-auth.json'
removed 'gcp-auth.json.1'
--2019-11-18 12:53:49--  http://host.romanr.info/gcp-auth.json
Resolving host.romanr.info (host.romanr.info)... 85.143.222.80, 2a04:ac00:1:52f1:5054:ff:fe00:6cc1
Connecting to host.romanr.info (host.romanr.info)|85.143.222.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2337 (2.3K) [application/json]
Saving to: ‘gcp-auth.json’

gcp-auth.json       100%[===================>]   2.28K  --.-KB/s    in 0s      

2019-11-18 12:53:50 (249 MB/s) - ‘gcp-auth.json’ saved [2337/2337]



In [0]:
from bs4 import BeautifulSoup
from google.cloud import translate
import pandas as pd
import time
import numpy as np

tr_cli = translate.Client()

htmlbatch = []
csvbatch = []
textbatch = []
request_size = 10000
def process_batch():
  global htmlbatch, csvbatch, textbatch
  if len(textbatch) == 0:
    return
  batch = []
  for i in range(len(textbatch)):
    text = textbatch[i]
    n = 0
    if len(text) > request_size:
      for k in range(request_size, len(text), request_size):
        batch.append((text[k-request_size:k], i, n))
        n += 1
      if len(text[k:]) > 0:
        batch.append((text[k:], i, n))
        n += 1
      assert(len([x for x in batch if x[1] == i]) > 1)
      assert(''.join([x[0] for x in batch if x[1] == i]) == text)
    else:
      batch.append((text, i, n))
  batches = []
  for job in batch:
    job_scheduled = False
    for b in batches:
      if len(''.join([j[0] for j in b])) + len(job[0]) <= request_size:
        b.append(job)
        job_scheduled = True
        break
    if not job_scheduled:
      batches.append([job])
  batch = [j for b in batches for j in b]
  trs = []
  for b in batches:
    assert(len(''.join([j[0] for j in b])) <= request_size)
    n = 1
    while True:
      try:
        trs += tr_cli.translate([j[0] for j in b], target_language='en', format_='text')
        break
      except Exception as e:
        print(e)
        time.sleep(n)
        n *= 2
  #assert(len(set([tr['detectedSourceLanguage'] for tr in trs])) > 1)
  for i in set([job[1] for job in batch]):
    idxs = np.array([k for k in range(len(batch)) if batch[k][1] == i])
    idxs = idxs[np.argsort([batch[k][2] for k in idxs])]
    assert(''.join([batch[k][0] for k in idxs]) == textbatch[i])
    trs_i = [trs[k] for k in idxs]
    df = pd.DataFrame()
    df['src'] = [textbatch[i]]
    df['lang'] = [','.join(set([tr['detectedSourceLanguage'] for tr in trs_i]))]
    df['text'] = [''.join([tr['translatedText'] for tr in trs_i])]
    df.to_csv(csvbatch[i])
  htmlbatch = []
  csvbatch = []
  textbatch = []

batch_size = 50
def process(htmlfn, csvfn):
  global htmlbatch, csvbatch, textbatch
  with open(htmlfn, 'r') as f:
    text = BeautifulSoup(f, 'html.parser').get_text()
  htmlbatch.append(htmlfn)
  csvbatch.append(csvfn)
  textbatch.append(text)
  if len(textbatch) > batch_size:
    process_batch()

In [27]:
from tqdm import tqdm
for i in tqdm(range(len(files))):
  csv = os.path.splitext(files[i])[0] + '.csv'
  if os.path.exists(csv):
    continue
  process(files[i], csv)
process_batch()




  0%|          | 0/166551 [00:00<?, ?it/s]


  9%|▉         | 15788/166551 [00:00<00:00, 157876.39it/s]


 21%|██        | 34679/166551 [00:00<00:00, 166058.84it/s]


 32%|███▏      | 53146/166551 [00:00<00:00, 171234.59it/s]


 43%|████▎     | 72336/166551 [00:00<00:00, 176949.80it/s]


 55%|█████▍    | 91554/166551 [00:00<00:00, 181257.93it/s]


 67%|██████▋   | 111293/166551 [00:00<00:00, 185813.40it/s]


 70%|███████   | 116807/166551 [00:17<00:44, 1106.67it/s]  


 70%|███████   | 116807/166551 [00:17<00:44, 1106.67it/s]


 70%|███████   | 116841/166551 [00:22<37:18, 22.21it/s]  


 70%|███████   | 116871/166551 [00:27<1:11:09, 11.63it/s]


 70%|███████   | 116922/166551 [00:34<1:21:35, 10.14it/s]


 70%|███████   | 116972/166551 [00:34<57:33, 14.36it/s]  


 70%|███████   | 116996/166551 [00:41<1:52:31,  7.34it/s]


 70%|███████   | 117024/166551 [00:48<2:22:32,  5.79it/s]


 70%|███████   | 117075/166551 [00:54<2:09:04,  6.39it/s]


 70%|███████   | 117126/166551 [01:01<2:04:

In [0]:
len(''.join(textbatch))

39005

In [18]:
[len(x) for x in textbatch]

[686,
 2126,
 993,
 4001,
 1946,
 897,
 969,
 2612,
 1450,
 1198,
 1980,
 62,
 4442,
 2296,
 2728,
 1301,
 2048,
 1418,
 263,
 1034,
 1118,
 898,
 10168,
 1412,
 621,
 14291,
 1489,
 1559,
 1449,
 159,
 1064,
 1975,
 4253,
 385,
 661,
 2376,
 5317,
 1936,
 1403,
 968,
 1129,
 5821,
 1887,
 1670,
 1207,
 1879,
 2359,
 4720,
 1100,
 6015,
 596]

In [19]:
textbatch

['\n\n\n\n\n\n\n\n\n\n\n\nسمو الأمير يهنئ ملك تونغا بالعيد الوطني لبلاده\n\n04 Nov 2019\n\nكونا - بعث حضرة صاحب السمو أمير البلاد الشيخ صباح الأحمد الجابر الصباح حفظه الله ورعاه ببرقية تهنئة إلى جلالة الملك توبو السادس ملك مملكة تونغا الصديقة عبر فيها سموه حفظه الله عن خالص تهانيه بمناسبة العيد الوطني لبلاده متمنيا سموه رعاه الله لجلالته موفور الصحة والعافية وللبلد الصديق دوام التقدم والازدهار.وبعث سمو ولي العهد الشيخ نواف الأحمد الجابر الصباح حفظه الله ببرقية تهنئة إلى جلالة الملك توبو السادس ملك مملكة تونغا الصديقة ضمنها سموه خالص تهانيه بمناسبة العيد الوطني لبلاده راجيا لجلالته موفور الصحة والعافية.كما بعث سمو الشيخ جابر المبارك الحمد الصباح رئيس مجلس الوزراء حفظه الله ببرقية تهنئة مماثلة.\n\n\n',
 '\n\n\n\n\n\n\n\n\n\n\n\nИркутские ученые включатся в работу на самом крупном в мире лазере на свободных электронах XFEL\nЕвропейский рентгеновский лазер на свободных электронах расположен в Германии под землей на глубине от 6 до 38 метров\n04 Nov 2019, 00:55 by Волшебный редактор\nУченые

In [0]:
tr_cli.translate(textbatch, target_language='en', format_='text')

[{'detectedSourceLanguage': 'de',
  'input': '\n\n\n\n\n\n\n\n\n\n\n\nKöster (SPD) zu griechischen Flüchtlingslagern"Der Druck auf die Staaten muss erhöht werden"\nEs fehle die politische Bereitschaft in der EU, zu einer menschenrechtsfundierten Flüchtlingspolitik zu kommen, sagte Dietmar Köster, SPD-Europaabgeordneter, im Dlf. Die Mitgliedsstaaten müssten stärker nach dem solidarischen Prinzip handeln und Griechenland bei der Bewältigung der Flüchtlingssituation helfen.\n02 Nov 2019 by Dietmar Köster im Gespräch mit\xa0Rainer Brandes\nRainer Brandes: Das griechische Parlament hat das Asylrecht drastisch verschärft: Flüchtlinge sollen schneller in die Türkei abgeschoben werden, wer nicht mit den Behörden kooperiert, dessen Antrag wird automatisch als unbegründet abgelehnt. Und: 20.000 Flüchtlinge sollen bis Ende des Jahres von den griechischen Inseln aufs Festland gebracht werden. Die Regierung will damit auch die Lage der Flüchtlinge auf den Insel entschärfen, die ist nämlich katastro

In [28]:
!find dataset1 -name '*.csv' | wc -l

166551


In [29]:
!rm -vf dataset1.zip
!zip -9rq dataset1.zip dataset1
!ls -l

removed 'dataset1.zip'
total 720972
drwxr-xr-x 9 root root      4096 Nov 16 16:11 dataset1
-rw-r--r-- 1 root root 738251231 Nov 18 18:33 dataset1.zip
-rw-r--r-- 1 root root      2337 Nov 18 12:24 gcp-auth.json
drwx------ 3 root root      4096 Nov 18 12:50 gdrive
drwxr-xr-x 1 root root      4096 Nov  6 16:17 sample_data


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [30]:
!cp -vf dataset1.zip /content/gdrive/*/

'dataset1.zip' -> '/content/gdrive/My Drive/dataset1.zip'


In [6]:
#!cp -v /content/gdrive/*/dataset1.zip ./
#!unzip -qu dataset1.zip
!ls -l

total 364204
drwxr-xr-x 9 root root      4096 Nov 16 16:11 dataset1
-rw------- 1 root root 372922779 Nov 18 12:50 dataset1.zip
-rw-r--r-- 1 root root      2372 Nov 17 18:14 gcp-auth.json
drwx------ 3 root root      4096 Nov 18 12:50 gdrive
drwxr-xr-x 1 root root      4096 Nov  6 16:17 sample_data
